In [ ]:
import requests
import json
import time
from tqdm import tqdm
from balance_date import BalanceDate
# 基本的数据处理库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import utils

# 获取结果函数

In [ ]:
def submit(sequence, change):
    """ 提交查询请求 """
    url = "http://old.protein.bio.unipd.it/sodaws/submitsoda"
    payload = (
        ("sequence",sequence.replace("X", "A")), 
        ("change[1]", change),
    )
    response = requests.post(url, data=payload)
    print("提交请求", response.text)
    response_json = json.loads(response.text)
    jobid = response_json["jobid"]
    return jobid


def query_status(jobid):
    """ 查询 jobid 的运行状态"""
    url_status = "http://protein.bio.unipd.it/sodaws/status/" + jobid
    response_status = requests.post(url_status)
    print("获取运行状态", response_status.text)
    response_json = json.loads(response_status.text)
    if "status" not in response_json.keys():
        response_json["status"] = "unknown"
    
    status = response_json["status"]
    return status

def get_output_table(jobid):
    """ 获取结果（表格格式）"""
    url_output = "http://protein.bio.unipd.it/sodaws/result/{}?format=tab".format(jobid)
    response_output_table = requests.post(url_output)
    print("获取结果", response_output_table.text)
    return response_output_table.text

def get_output_detail(jobid):
    """ 获取结果（表格格式）"""
    url_output = "http://protein.bio.unipd.it/sodaws/result/{}?l=false".format(jobid)
    response_output_table = requests.post(url_output)
    response_json = json.loads(response_output_table.text)
    print("获取结果", "difference:", response_json["parsed_soda_output"]["1"]["difference"])
    return response_json

In [ ]:
def soda_predict(seq, chg):
    # 1. 提交请求
    jobid = submit(seq, chg)

    # 2. 判断任务是否完成, timeout = 5 * 10 s
    stat = query_status(jobid)
    counter = 0
    while(stat != "done"):
        if counter == 0:
            time.sleep(1)  # 针对第一次少休息一会会
        elif counter > 100:
            break
        else:
            time.sleep(5)  # 如果还没有运行完成，休息 10 s
        stat = query_status(jobid)
        counter += 1
    if stat != "done":
        raise Exception("未成功获取数据")

    # 3. 获取结果
    # 获取结果(表格)
    #     get_output_table(jobid)
    # 获取详细格式
    diff = get_output_detail(jobid)["parsed_soda_output"]["1"]["difference"]

    return diff
   

In [ ]:
# 测试
seq = "NVSIKFQALD"
chg = "V2S"
print(soda_predict(seq, chg))

# 读取数据

In [ ]:
# 定义常量
OUT_PATH = os.path.join(".", "out", time.strftime("%Y%m%d"))  # 输出路径
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)
print("输出路径为：", OUT_PATH)

In [ ]:
config_path = "./config.json"
with open(config_path) as f:
    config = json.loads(f.read())
config

In [ ]:
# 获取基因序列文件
genes = pd.read_csv(config["genes_seq"])
genes

In [ ]:
# 获取盲测集
X_test1, y_test1, X_test2, y_test2 = BalanceDate.get_test()  # 两组测试数据  
X_test1 = pd.merge(X_test1, genes, left_on="gi", right_on="gi").loc[:, ["index", "gi", "solubility", "variation", "seq"]]
X_test2 = pd.merge(X_test2, genes, left_on="gi", right_on="gi").loc[:, ["index", "gi", "solubility", "variation", "seq"]]
# print(X_test1.to_string())
X_test1
X_test2

# 训练

In [ ]:
X_test1["soda"] = X_test1.apply(lambda x: soda_predict(x["seq"], x["variation"]), axis = 1)
X_test1

In [ ]:
"""
提交请求 {"status":"submitted","itype":"multi","jobid":"60029ea58fbfe2840bca0dca"}
获取运行状态 {"jobid":"60029ea58fbfe2840bca0dca"}
获取运行状态 {"status":"error","jobid":"60029ea58fbfe2840bca0dca","error":["Impossible to submit session:60029ea58fbfe2840bca0dca, please check input sequence or file"]}
"""
X_test2["soda"] = X_test2.apply(lambda x: soda_predict(x["seq"], x["variation"]), axis = 1)
X_test2

In [ ]:
_test1_path = os.path.join(OUT_PATH, "soda_test1_result.xlsx")
_test2_path = os.path.join(OUT_PATH, "soda_test2_result.xlsx")
X_test1.to_excel(_test1_path)
X_test2.to_excel(_test2_path)

print("输出路径为：")
print(_test1_path)
print(_test2_path)

# 评估

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, matthews_corrcoef, roc_curve, auc, make_scorer, mean_squared_error, confusion_matrix
def assess_detail(y_true, y_pred, kinds=[-1, 1]):
    """ 二分类
    预测每一类的 tp 等
    :param y_true: 真实值
    :param y_pred: 预测值
    :param kinds: 具体的组别
    """
    y_true = pd.Series(y_true)
    y_pred = pd.Series(y_pred)
    print("discribution:")
    print("true:", pd.value_counts(y_true).to_dict())
    print("pred:", pd.value_counts(y_pred).to_dict())
    N = len(y_true)  # 数据的数目
    matrixs = []
    for i in kinds:
        tmp_y_true = y_true.apply(lambda x: 1 if x == i else 0)
        tmp_y_pred = y_pred.apply(lambda x: 1 if x == i else 0)
        tmp_res = confusion_matrix(tmp_y_true, tmp_y_pred).ravel()
        matrixs.append(tmp_res)
    # TP FN FP TN
    res = pd.DataFrame(matrixs, columns="tn fp fn tp".split(), index=kinds)

    # PPV NPV TPR TNR
    res["ppv"] = res.tp / (res.tp + res.fp)
    res["npv"] = res.tn / (res.tn + res.fn)
    res["sensitivity"] = res.tp / (res.tp + res.fn)
    res["specificity"] = res.tn / (res.tn + res.fp)

    # gcc
    # zij, xi, yi
    zij = confusion_matrix(y_true, y_pred, labels=kinds).astype(np.float)

    xi = np.sum(zij, axis=1)  # 原始值为 i 的数目
    yi = np.sum(zij, axis=0)  # 预测值为 j 的数目
    # xij, yij
    xij = xi.reshape(xi.shape[0], 1)
    xij = np.repeat(xij, xi.shape[0]).reshape((-1, xi.shape[0]))
    yij = np.array([yi for _ in range(yi.shape[0])])
    # zij
    eij = xij * yij / N
    gcc = np.sum((zij - eij) ** 2 / eij) / (N * (len(kinds) - 1))
    acc = accuracy_score(y_true, y_pred)
    res["gcc"] = gcc
    res["acc"] = acc
    return res

In [ ]:
names = ["variation", "gi", "unknow", "score", "predict"]

def tran_to_ponsol1(dataset):
    """将数据格式转化为 PON-Sol 2016需要的格式"""
    columns = "variation  gi solubility  sift len.1 ANDN920101 ARGP820101 ARGP820102 ARGP820103 BEGF750101 BEGF750102 BEGF750103 BHAR880101 BIGC670101 BIOV880101 BIOV880102 BROC820101 BROC820102 BULH740101 BUNA790102 BURA740101 BURA740102 CHAM820101 CHAM830101 CHOC760101 CHOC760102 CHOC760103 CHOC760104 CHOP780101 CHOP780201 CHOP780202 CHOP780203 CHOP780204 CHOP780205 CHOP780206 CHOP780207 CHOP780208 CHOP780209 CHOP780210 CHOP780211 CHOP780212 CHOP780213 CHOP780214 CHOP780215 CHOP780216 CIDH920101 CIDH920102 CIDH920103 CIDH920104 CIDH920105 COHE430101 CRAJ730101 CRAJ730102 CRAJ730103 DAWD720101 DAYM780101 DAYM780201 DESM900101 DESM900102 EISD840101 EISD860101 EISD860102 FASG760101 FASG760102 FASG760103 FASG760104 FASG760105 FAUJ830101 FAUJ880101 FAUJ880106 FAUJ880107 FAUJ880109 FAUJ880110 FAUJ880111 FAUJ880112 FINA910101 FINA910103 FINA910104 GARJ730101 GEIM800101 GEIM800102 GEIM800103 GEIM800104 GEIM800105 GEIM800106 GEIM800107 GEIM800108 GEIM800109 GEIM800110 GEIM800111 GOLD730102 GRAR740101 GRAR740102 GRAR740103 GUYH850101 HOPT810101 HUTJ700101 HUTJ700102 HUTJ700103 ISOY800101 ISOY800102 ISOY800103 ISOY800104 ISOY800105 ISOY800106 ISOY800107 ISOY800108 JANJ780101 JANJ780102 JANJ780103 JANJ790101 JANJ790102 JOND750101 JOND750102 JOND920101 JOND920102 JUKT750101 JUNJ780101 KANM800101 KANM800102 KANM800103 KANM800104 KARP850101 KARP850102 KARP850103 KLEP840101 KRIW710101 KRIW790101 KRIW790102 KRIW790103 KYTJ820101 LAWE840101 LEVM760101 LEVM760102 LEVM760104 LEVM760105 LEVM760106 LEVM760107 LEVM780101 LEVM780102 LEVM780103 LEVM780104 LEVM780105 LEVM780106 LEWP710101 LIFS790101 LIFS790102 LIFS790103 MANP780101 MAXF760101 MAXF760102 MAXF760103 MAXF760104 MAXF760105 MAXF760106 MCMT640101 MEEJ800101 MEEJ800102 MEEJ810101 MEEJ810102 MEIH800101 MEIH800102 MEIH800103 MIYS850101 NAGK730101 NAGK730102 NAGK730103 NAKH900101 NAKH900102 NAKH900103 NAKH900104 NAKH900105 NAKH900106 NAKH900107 NAKH900108 NAKH900109 NAKH900110 NAKH900111 NAKH900112 NAKH900113 NAKH920101 NAKH920102 NAKH920103 NAKH920104 NAKH920105 NAKH920106 NAKH920107 NAKH920108 NISK800101 NISK860101 OOBM770101 OOBM770102 OOBM770103 OOBM770104 OOBM770105 OOBM850101 OOBM850102 OOBM850103 OOBM850104 OOBM850105 PALJ810101 PALJ810102 PALJ810103 PALJ810104 PALJ810105 PALJ810106 PALJ810107 PALJ810108 PALJ810109 PALJ810110 PALJ810111 PALJ810112 PALJ810115 PALJ810116 PARJ860101 PLIV810101 PONP800101 PONP800102 PONP800103 PONP800104 PONP800105 PONP800106 PONP800107 PONP800108 PRAM820101 PRAM820102 PRAM820103 PRAM900101 PRAM900102 PRAM900103 PRAM900104 PTIO830101 PTIO830102 QIAN880101 QIAN880102 QIAN880103 QIAN880104 QIAN880105 QIAN880106 QIAN880107 QIAN880108 QIAN880109 QIAN880110 QIAN880111 QIAN880112 QIAN880113 QIAN880114 QIAN880115 QIAN880116 QIAN880117 QIAN880118 QIAN880119 QIAN880120 QIAN880121 QIAN880122 QIAN880123 QIAN880124 QIAN880125 QIAN880126 QIAN880127 QIAN880128 QIAN880129 QIAN880130 QIAN880131 QIAN880132 QIAN880133 QIAN880134 QIAN880136 QIAN880137 QIAN880138 QIAN880139 RACS770101 RACS770102 RACS820101 RACS820102 RACS820103 RACS820104 RACS820105 RACS820106 RACS820107 RACS820108 RACS820109 RACS820110 RACS820111 RACS820112 RACS820113 RACS820114 RICJ880101 RICJ880102 RICJ880103 RICJ880104 RICJ880105 RICJ880106 RICJ880107 RICJ880108 RICJ880109 RICJ880110 RICJ880111 RICJ880112 RICJ880113 RICJ880114 RICJ880115 RICJ880116 RICJ880117 ROBB760101 ROBB760102 ROBB760103 ROBB760104 ROBB760105 ROBB760106 ROBB760107 ROBB760108 ROBB760109 ROBB760110 ROBB760111 ROBB760112 ROBB760113 ROBB790101 ROSG850101 ROSG850102 ROSM880101 ROSM880102 ROSM880103 SIMZ760101 SNEP660101 SNEP660102 SNEP660103 SNEP660104 SUEM840101 SWER830101 TANS770101 TANS770102 TANS770103 TANS770104 TANS770105 TANS770106 TANS770107 TANS770108 TANS770109 TANS770110 VASM830101 VASM830102 VELV850101 VENT840101 VHEG790101 WARP780101 WEBA780101 WERD780101 WERD780102 WERD780104 WOEC730101 WOLR810101 WOLS870101 WOLS870102 WOLS870103 ZIMJ680101 ZIMJ680102 ZIMJ680103 ZIMJ680104 ZIMJ680105 AURR980101 AURR980102 AURR980103 AURR980104 AURR980105 AURR980106 AURR980107 AURR980108 AURR980109 AURR980110 AURR980111 AURR980112 AURR980113 AURR980114 AURR980115 AURR980116 AURR980117 AURR980118 AURR980119 AURR980120 ONEK900101 ONEK900102 VINM940101 VINM940102 VINM940103 VINM940104 MUNV940101 MUNV940102 MUNV940103 MUNV940104 MUNV940105 WIMW960101 KIMC930101 MONM990101 BLAM930101 PARS000101 PARS000102 KUMS000101 KUMS000102 KUMS000103 KUMS000104 TAKK010101 FODM020101 NADH010101 NADH010102 NADH010103 NADH010104 NADH010105 NADH010106 NADH010107 MONM990201 CEDJ970101 CEDJ970102 CEDJ970103 CEDJ970104 CEDJ970105 FUKS010101 FUKS010102 FUKS010103 FUKS010104 FUKS010105 FUKS010106 FUKS010107 FUKS010108 FUKS010109 FUKS010110 FUKS010111 FUKS010112 MITS020101 TSAJ990101 TSAJ990102 COSI940101 PONP930101 WILM950101 WILM950102 WILM950103 WILM950104 KUHL950101 GUOD860101 JURD980101 BASU050101 BASU050102 BASU050103 SUYM030101 PUNT030101 PUNT030102 GEOR030101 GEOR030102 GEOR030103 GEOR030104 GEOR030105 GEOR030106 GEOR030107 GEOR030108 GEOR030109 ZHOH040101 ZHOH040102 ZHOH040103 BAEK050101 HARY940101 PONJ960101 DIGM050101 WOLR790101 OLSK800101 KIDA850101 GUYH850102 GUYH850104 GUYH850105 JACR890101 COWR900101 BLAS910101 CASG920101 CORJ870101 CORJ870102 CORJ870103 CORJ870104 CORJ870105 CORJ870106 CORJ870107 CORJ870108 MIYS990101 MIYS990102 MIYS990103 MIYS990104 MIYS990105 ENGD860101 FASG890101 ALTS910101 BENS940101 BENS940102 BENS940103 BENS940104 CSEM940101 DAYM780301 FEND850101 FITW660101 GEOD900101 GONG920101 GRAR740104 HENS920101 HENS920102 HENS920103 JOHM930101 JOND920103 JOND940101 KOLA920101 LEVJ860101 LUTR910101 LUTR910102 LUTR910103 LUTR910104 LUTR910105 LUTR910106 LUTR910107 LUTR910108 LUTR910109 MCLA710101 MCLA720101 MIYS930101 MIYT790101 MOHR870101 NIEK910101 NIEK910102 OVEJ920101 QU_C930101 QU_C930102 QU_C930103 RISJ880101 TUDE900101 AZAE970101 AZAE970102 RIER950101 WEIL970101 WEIL970102 MEHP950102 KAPO950101 VOGG950101 KOSJ950101 KOSJ950102 KOSJ950103 KOSJ950104 KOSJ950105 KOSJ950106 KOSJ950107 KOSJ950108 KOSJ950109 KOSJ950110 KOSJ950111 KOSJ950112 KOSJ950113 KOSJ950114 KOSJ950115 OVEJ920102 OVEJ920103 OVEJ920104 OVEJ920105 LINK010101 BLAJ010101 PRLA000101 PRLA000102 DOSZ010101 DOSZ010102 DOSZ010103 DOSZ010104 GIAG010101 DAYM780302 HENS920104 QUIB020101 NAOD960101 RUSR970101 RUSR970102 RUSR970103 OGAK980101 KANM000101 NGPC000101 MUET010101 MUET020101 MUET020102 CROG050101 TANS760101 TANS760102 BRYS930101 THOP960101 MIRL960101 VENM980101 BASU010101 MIYS850102 MIYS850103 MIYS960101 MIYS960102 MIYS960103 MIYS990106 MIYS990107 LIWA970101 KESO980101 KESO980102 MOOG990101 BETM990101 TOBD000101 TOBD000102 KOLA930101 SKOJ970101 SKOJ000101 SKOJ000102 BONM030101 BONM030102 BONM030103 BONM030104 BONM030105 BONM030106 MICC010101 SIMK990101 SIMK990102 SIMK990103 SIMK990104 SIMK990105 ZHAC000101 ZHAC000102 ZHAC000103 ZHAC000104 ZHAC000105 ZHAC000106 A.A  A.C  A.D  A.E  A.F  A.G  A.H  A.I  A.K  A.L  A.M  A.N  A.P  A.Q  A.R  A.S  A.T  A.V  A.W  A.Y  C.A  C.C  C.D  C.E  C.F  C.G  C.H  C.I  C.K  C.L  C.M  C.N  C.P  C.Q  C.R  C.S  C.T  C.V  C.W  C.Y  D.A  D.C  D.D  D.E  D.F  D.G  D.H  D.I  D.K  D.L  D.M  D.N  D.P  D.Q  D.R  D.S  D.T  D.V  D.W  D.Y  E.A  E.C  E.D  E.E  E.F  E.G  E.H  E.I  E.K  E.L  E.M  E.N  E.P  E.Q  E.R  E.S  E.T  E.V  E.W  E.Y  F.A  F.C  F.D  F.E  F.F  F.G  F.H  F.I  F.K  F.L  F.M  F.N  F.P  F.Q  F.R  F.S  F.T  F.V  F.W  F.Y  G.A  G.C  G.D  G.E  G.F  G.G  G.H  G.I  G.K  G.L  G.M  G.N  G.P  G.Q  G.R  G.S  G.T  G.V  G.W  G.Y  H.A  H.C  H.D  H.E  H.F  H.G  H.H  H.I  H.K  H.L  H.M  H.N  H.P  H.Q  H.R  H.S  H.T  H.V  H.W  H.Y  I.A  I.C  I.D  I.E  I.F  I.G  I.H  I.I  I.K  I.L  I.M  I.N  I.P  I.Q  I.R  I.S  I.T  I.V  I.W  I.Y  K.A  K.C  K.D  K.E  K.F  K.G  K.H  K.I  K.K  K.L  K.M  K.N  K.P  K.Q  K.R  K.S  K.T  K.V  K.W  K.Y  L.A  L.C  L.D  L.E  L.F  L.G  L.H  L.I  L.K  L.L  L.M  L.N  L.P  L.Q  L.R  L.S  L.T  L.V  L.W  L.Y  M.A  M.C  M.D  M.E  M.F  M.G  M.H  M.I  M.K  M.L  M.M  M.N  M.P  M.Q  M.R  M.S  M.T  M.V  M.W  M.Y  N.A  N.C  N.D  N.E  N.F  N.G  N.H  N.I  N.K  N.L  N.M  N.N  N.P  N.Q  N.R  N.S  N.T  N.V  N.W  N.Y  P.A  P.C  P.D  P.E  P.F  P.G  P.H  P.I  P.K  P.L  P.M  P.N  P.P  P.Q  P.R  P.S  P.T  P.V  P.W  P.Y  Q.A  Q.C  Q.D  Q.E  Q.F  Q.G  Q.H  Q.I  Q.K  Q.L  Q.M  Q.N  Q.P  Q.Q  Q.R  Q.S  Q.T  Q.V  Q.W  Q.Y  R.A  R.C  R.D  R.E  R.F  R.G  R.H  R.I  R.K  R.L  R.M  R.N  R.P  R.Q  R.R  R.S  R.T  R.V  R.W  R.Y  S.A  S.C  S.D  S.E  S.F  S.G  S.H  S.I  S.K  S.L  S.M  S.N  S.P  S.Q  S.R  S.S  S.T  S.V  S.W  S.Y  T.A  T.C  T.D  T.E  T.F  T.G  T.H  T.I  T.K  T.L  T.M  T.N  T.P  T.Q  T.R  T.S  T.T  T.V  T.W  T.Y  V.A  V.C  V.D  V.E  V.F  V.G  V.H  V.I  V.K  V.L  V.M  V.N  V.P  V.Q  V.R  V.S  V.T  V.V  V.W  V.Y  W.A  W.C  W.D  W.E  W.F  W.G  W.H  W.I  W.K  W.L  W.M  W.N  W.P  W.Q  W.R  W.S  W.T  W.V  W.W  W.Y  Y.A  Y.C  Y.D  Y.E  Y.F  Y.G  Y.H  Y.I  Y.K  Y.L  Y.M  Y.N  Y.P  Y.Q  Y.R  Y.S  Y.T  Y.V  Y.W  Y.Y  g1.g1  g1.g2  g1.g3  g1.g4  g1.g5  g1.g6  g2.g1  g2.g2  g2.g3  g2.g4  g2.g5  g2.g6  g3.g1  g3.g2  g3.g3  g3.g4  g3.g5  g3.g6  g4.g1  g4.g2  g4.g3  g4.g4  g4.g5  g4.g6  g5.g1  g5.g2  g5.g3  g5.g4  g5.g5  g5.g6  g6.g1  g6.g2  g6.g3  g6.g4  g6.g5  g6.g6  AA20D.A  AA20D.C  AA20D.D  AA20D.E  AA20D.F  AA20D.G  AA20D.H  AA20D.I  AA20D.K  AA20D.L  AA20D.M  AA20D.N  AA20D.P  AA20D.Q  AA20D.R  AA20D.S  AA20D.T  AA20D.V  AA20D.W  AA20D.Y  NonPolarAA  PolarAA  ChargedAA  PosAA  NegAA".strip().split()
    rename_columns = {
        "gi": "protein",
        "sift": "conservation", 
        "len.1": "numberofseq",
    }

    dataset_re = dataset.loc[:, columns].rename(columns=rename_columns)
    return dataset_re

def save_ponsol1_dataset(dataset, file_name):
    """保存结果"""
    dataset.to_csv(os.path.join(OUT_PATH, file_name + ".traindata"), sep=" ", index=False)


def generate_output(y_true, y_pred):
    """生成报告"""
    def combine_res(acc, gc2, metr):
        res = metr.unstack()
        res = res.append(
            pd.Series([acc, gc2], index=[['all', 'all'], ['accuracy', 'gc2']]))
        return res


    acc, gc2, metr = utils.ponsol_metrics(y_true, y_pred)
    res1 = combine_res(acc, gc2, metr)  # unbalance

    acc, gc2, metr = utils.ponsol_metrics(y_true, y_pred, balance=True)
    res2 = combine_res(acc, gc2, metr)  # balance
    res_all = pd.concat([res1, res2], axis=1)
    res_all

    res = []
    for i in range(len(res1)):
        if i >=12:
            res.append('%.3f/%.3f' % (res1.iloc[i], res2.iloc[i]))
        else: 
            res.append('%.1f/%.1f' % (res1.iloc[i], res2.iloc[i]))
            
    res = pd.Series(res, index=res1.index)
    return res

In [ ]:
res_test1 = pd.read_excel("./out/20210209/soda_test1_result.xlsx", ).iloc[:, 1:]
res_test2 = pd.read_excel("./out/20210209/soda_test2_result.xlsx", ).iloc[:, 1:]
res_test2

## 三分类

In [ ]:
test1_true = res_test1["solubility"]
test1_pred = res_test1["soda"].apply(lambda x: 1 if x > 0 else -1)
print(utils.ponsol_metrics(test1_true, test1_pred))
# utils.ponsol_metrics(test1_true, test1_pred)[2]

In [ ]:
test2_true = res_test2["solubility"]
test2_pred = res_test2["soda"].apply(lambda x: 1 if x > 0 else -1)
print(utils.ponsol_metrics(test2_true, test2_pred))

In [ ]:
res_3_class = pd.concat([generate_output(test1_true, test1_pred),  generate_output(test2_true, test2_pred)], axis=1)
res_3_class.columns = ["test1", "test2"]
res_3_class.to_excel(os.path.join(OUT_PATH, "blind_res_soda.xlsx"))
res_3_class

## 三分类-设置阈值

In [ ]:
soda_threshold = 5
def split_classification(th, x):
    th = abs(th)
    if x >= th:
        return 1
    elif x <= -1 * th:
        return -1
    else:
        return 0
test1_true = res_test1["solubility"]
test1_soda = res_test1["soda"]
test1_pred = test1_soda.apply(lambda x: split_classification(soda_threshold, x))

test2_true = res_test2["solubility"]
test2_soda = res_test2["soda"]
test2_pred = test2_soda.apply(lambda x: split_classification(soda_threshold, x))

In [ ]:
res_3_class_th = pd.concat([generate_output(test1_true, test1_pred),  generate_output(test2_true, test2_pred)], axis=1)
res_3_class_th.columns = ["test1", "test2"]
res_3_class_th.to_excel(os.path.join(OUT_PATH, "blind_res_soda_threshold_{}.xlsx".format(soda_threshold)))
res_3_class_th

## 二分类

In [ ]:
test1_true = res_test1[res_test1["solubility"] != 0]["solubility"]
test1_pred = res_test1[res_test1["solubility"] != 0]["soda"].apply(lambda x: 1 if x > 0 else -1)
# ponsol_metrics(test1_true, test1_pred)
assess_detail(test1_true, test1_pred, kinds=[-1, 1])

In [ ]:
test2_true = res_test2[res_test2["solubility"] != 0]["solubility"]
test2_pred = res_test2[res_test2["solubility"] != 0]["soda"].apply(lambda x: 1 if x > 0 else -1)
assess_detail(test2_true, test2_pred, kinds=[-1, 1])